## Aircraft Detection - Inference

In [ ]:
import torch
from bbox_utils import *
import torch
from torch import nn
import torch.optim as optimizer
import torch.nn.functional as f
from pathlib import Path
import os
import bbox_utils
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Conv2d, Dropout, MaxPool2d, BatchNorm1d, BatchNorm2d
import pandas as pd
import json
import visualization
import model

In [ ]:
new_size = 128
ratio = int(512/new_size)

def resize_bbox_rowwise(row):
    return np.array(bbox_utils.transformsBbox(row['bbox'], ratio), dtype='float64')

def resize_im_rowwise(row):
    return bbox_utils.transformsImg(row['path'], new_size)

In [ ]:
#Loading the model

dir_root = os.path.abspath('')
model_dir = Path(dir_root, '../data/model/2022-12-08 11_02_25/model/epoch_119.pth') # replace here the name of the model you want to use

In [ ]:
train_imgs = Path(dir_root, '../data/train')
test_imgs = Path(dir_root, '../data/test')
annot_dir = Path(dir_root, '../data/annot/rareplanes.json')

In [ ]:
with open(annot_dir) as json_data:
    data = json.load(json_data)

annot_train = pd.DataFrame(data['categories'])
annot_train.drop(['loc_id', 'cat_id', 'location', 'role', 'role_id', 'is_plane', 'num_engines', 'propulsion', 'canards', 'num_tail_fins', 'wing_position', 'wing_type',
                'length', 'wingspan', 'area', 'faa_wingspan_class', 'Public_Train', 'Public_Test', 'partialDec', 'truncated', 'new_area', 'area_pixels', 'id'], axis=1, inplace=True)
annot_train.rename(columns={"image_fname": "name"}, inplace=True)

annot_train = annot_train.groupby(['image_id']).agg(
    tuple).applymap(np.array).reset_index()

annot_train.drop(annot_train.index.to_list()[100:], axis=0, inplace=True)

annot_train['path'] = annot_train.apply(
    lambda row: str(train_imgs) + "/"+row['name'][0], axis=1)
annot_train.drop(['name', 'image_id'], axis=1, inplace=True)

In [ ]:
annot_train['image'] = annot_train.apply(resize_im_rowwise, axis=1)
annot_train['bbox'] = annot_train.apply(resize_bbox_rowwise, axis=1)
annot_train.head()

In [ ]:
plt.figure()
plt.imshow(annot_train['image'][78])
plt.show()

In [ ]:
DEBUG_LAYERS = False

aircraft_model = model.AircraftModel().double()
loaded_model = torch.load(model_dir, map_location = torch.device('cpu'))

if DEBUG_LAYERS:
    print('------------ MODEL FORMAT -------------')
    for layer_name, tensor in aircraft_model.named_parameters():
        print(layer_name, tensor.size())
    print('----------- LOADED MODEL --------------')
    for layer_name, tensor in loaded_model.named_parameters():
        print(layer_name, tensor.size())


#model.load_state_dict(loaded_model.state_dict())
aircraft_model.load_state_dict(loaded_model)
aircraft_model.eval()

In [ ]:
import training_utilities
import random

# import importlib
# importlib.reload(training_utilities)

for i in random.sample(range(0, len(annot_train['image'])), 10):

    image_index = i
    # Confidence equal to, or higher gets shown
    THRESHHOLD = 0.5

    inputs = torch.from_numpy(np.array([annot_train['image'][image_index]]))
    inputs = torch.permute(inputs, (0, 3, 1, 2))

    output = aircraft_model(inputs)
    targets = training_utilities.calculate_iou(annot_train['bbox'][image_index])
    loss = training_utilities.loss_fn(output, torch.from_numpy(np.array([targets])))

    print('Loss', loss.item())
    plane_count = (output.detach().numpy() >= THRESHHOLD).sum()

    # print("Output: ",output)
    print('expected planes: ', targets.sum())
    print("Planes found: ", plane_count) #when the model outputs proper 1's and 0'2 this should make more sense

    visualization.display_bbox_target_vector(annot_train['image'][image_index], output[0], model.np_bboxs, THRESHHOLD)